##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications

# 1. Load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train, y_test = y_train.squeeze().astype("int64"), y_test.squeeze().astype("int64")
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")

# 2. Data Augmentation (Identical to ResNet reference)
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

# 3. Build MobileNetV2 Backbone
mobilenet_base = applications.MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3)
)
mobilenet_base.trainable = False  # Initially frozen

# 4. Full Model Construction
model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(applications.mobilenet_v2.preprocess_input),
    mobilenet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)
], name="cifar10_mobilenetv2_comparison")

# --- ARCHITECTURE INSPECTION (Requirement 2) ---
print("--- Architecture Observation ---")
print(f"Total layers in backbone (Network Depth): {len(mobilenet_base.layers)}")
model.summary() # Displays Number of parameters and Trainable vs Frozen layers
# -----------------------------------------------

# 5. Phase 1: Feature Extraction (Frozen Backbone)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

print("\nStarting Phase 1: Feature Extraction...")
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

# 6. Phase 2: Fine-Tuning (Unfreezing top layers)
mobilenet_base.trainable = True
for layer in mobilenet_base.layers[:-30]: # Unfreeze last 30 layers like ResNet lab
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

print("\nStarting Phase 2: Fine-Tuning...")
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

# --- FINAL EVALUATION (Requirement 3) ---
print("\n--- Final Model Evaluation ---")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f"MobileNetV2 Final Test Accuracy: {test_acc:.4f}")
print(f"MobileNetV2 Final Test Loss: {test_loss:.4f}")

In [3]:
# Evaluation
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n MobileNetV2 Test Accuracy: {test_acc:.4f}")